# Introduction to _Alhazen-Py_

The initial idea for writing this guide was to explain to our students at Humboldt-Universität Zu Berlin how to use _Alhazen_ to determine the failure circumstances of a program. The original notebooks were a joint project with my colleague [Hoang Lam Nguyen](https://www.informatik.hu-berlin.de/en/Members/hoang-lam-nguyen) from Humboldt-Universität Zu Berlin.

Considering the difficulties of determining the circumstances of a program’s behavior, Kampmann et al. [[KHSZ20](https://publications.cispa.saarland/3107/7/fse2020-alhazen.pdf)] presented an approach to automatically learn the associations between the failure of a program and the input data. Their tool _Alhazen_ affiliates specific syntactical input features, like input length or the presence of particular derivation sequences, with the behavior in question. This allows _Alhazen_ to hypothesize why failure-inducing input files result in a defect. In this notebook, we will extend _Alhazen_ [[KHSZ20](https://publications.cispa.saarland/3107/7/fse2020-alhazen.pdf)] and build our own framework to determine and explain the failure of a program.

To run this notebook, you should have the following Python packages installed. Please refer to the README.md of this project to get a complete list of requirements and instructions on how to install them.

- pandas, numpy
- fuzzingbook
- jupyter-notebook
- sklearn
- graphviz

<div class="alert alert-info">
[Tip]: To execute the Python code in the code cell below, click on the cell to select it and press <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>

In [ ]:
# To import other notebooks we need this dependency
# !pip install ipynb fuzzingbook pandas numpy sklearn

In [ ]:
# Verbose output
# Set to True to see some intermediate results
display_output = True

## Overview of Alhazen-Py

![title](img/overview.png)

Recent testing techniques, like fuzzing [[MFS90](https://dl.acm.org/doi/pdf/10.1145/96267.96279), [FMEH20](https://www.usenix.org/system/files/woot20-paper-fioraldi.pdf)], generate random input data and enhance or mutate them to trigger potential defects or software vulnerabilities. Although they have proven capable of detecting and generating erroneous input data, they often lack an explanation of why specific input data results in incorrect behavior. However, when diagnosing why a program fails, the first step is determining the circumstances under which the program failed. Kampmann et al. [[KHSZ20](https://publications.cispa.saarland/3107/7/fse2020-alhazen.pdf)] presented an approach to automatically discover the circumstances of program behavior. Their method associates the program’s failure with the syntactical features of the input data, allowing them to learn and extract the properties that result in the specific behavior. As a result, their tool Alhazen can generate a diagnosis and explain why, for instance, a particular bug occurs. More formally, Alhazen forms a hypothetical model based on the observed inputs. Then, additional test inputs are generated and executed to refine or refute the hypothesis, eventually obtaining a prediction model of why the behavior in question occurs. Alhazen uses a Decision Tree classifier to learn the association between the program behavior and the input features.

This collection of notebooks contain a reduced implementation **alhazen-py** that we will complete together.

### **To complete the feedback loop of alhazen-py we will look at the following individual tasks:**

**Activity 1:** Initial Setup and Feature Extraction 

2. Task: Write the functions `extract_existence`, `extract_numeric` and `collect_features` (FeatureExtraction.ipynb)
3. Task: Write the function `transform_grammar` (TransformGrammar.ipynb)

**Activity 2:** Train Classification Model

4. Task: Write a function `train_tree` (DecisionTreeLearner.ipynb)

**Activity 5:** Generate new Inputs Files

5. Task: Write a function `generate_samples` (GenerateSamples.ipynb)


(Please refer to the individual notebooks for a detailed description of the individual tasks.)

### Paper: Read the paper of Kampmann et al.

<div class="alert alert-success alertsuccess">
[Task] Before you continure, you should take a first look at intial <a href="https://publications.cispa.saarland/3107/7/fse2020-alhazen.pdf">paper</a> to understand the proposed approach and underlying concepts.
</div>

# Introduction
<hr/>

To illustrate the use-case and the necessity of _Alhazen_, we start with a quick motivating example.

## Motivating Example

To test _Alhazen_, we implemented the simple CALCULATOR example from the paper. We furthermore implemented a synthetic `BUG` that you have to explain utilizing the decision tree learned by _Alhazen_.

<div class="alert alert-danger" role="alert">
To make the bug explanation a little bit more challenging, we altered to calculator behavior. The introduced bug of Kampmann et al. is not the same as ours.
</div>

The calculator takes an input file as input and returns whether the bug was present or not (`BUG,` `NO_BUG`). Input files for the calculator have to conform to the `CALCULATOR`- Grammar. Let us have a look at the grammar definition:

In [ ]:
# Lets load the grammar from the calculator
from alhazen_formalizations.calculator import grammar_alhazen as grammar

for rule in grammar:
    print(rule.ljust(20), grammar[rule])

We see that the `CALCULATOR` Grammar consists of several production rules. The calculator subject will only accept inputs that conform to this grammar definition.

<div class="alert alert-info">
[Info]: We use the functionallity provided by <a href="https://www.fuzzingbook.org">The Fuzzingbook</a>. For a more detailed description of Grammars, have a look at the chapter <a href="https://www.fuzzingbook.org/html/Grammars.html">Fuzzing with Grammars</a>.
</div>

Now, lets load the initial input samples:

In [ ]:
from alhazen_formalizations.calculator import initial_inputs

display(initial_inputs)

The two initial imput samples for our calculator should be:
- _cos(12)_
- _sqrt(-900)_

Lets check if this is true with python's `assert` function. The condition is True, if no Assertion is thrown.

In [ ]:
# First sample
assert initial_inputs[0] == 'cos(12)', f"The loaded sample {sample_list[0]} and cos(12) are not equal."

# Second sample
assert initial_inputs[1] == 'sqrt(-900)', f"The loaded sample {sample_list[1]} and sqrt(-900) are not equal."

In [ ]:
# We intentionally throw an assertion error. sample_list[0] is equal to sqrt(-16)
try:
    assert initial_inputs[0] == 'cos(4)', f"The loaded sample {sample_list[0]} and cos(4) are not equal."
except AssertionError as e: 
    print("Expected Error: " + str(e))

Now, let's execute our two input samples and observe the calculator's behavior. To do this, we load the function `execute_samples` from the notebook ExecuteSamples.ipynb. We can call the function with a list of input samples, and it returns the corresponding execution outcome (label/oracle). The output is a [pandas dataframe](https://pandas.pydata.org/docs/reference/frame.html), and the labels are from the class `OracleResults`.

In [ ]:
# Load function execute samples
from ipynb.fs.defs.Activity5_ExecuteSamples import execute_samples

# execute_samples(List[str])
oracle = execute_samples(initial_inputs)
if display_output: display(oracle)

<div class="alert alert-info">
[Info]: For a more detailed description of the functionallity, you can look into the implementation of <i>execute_samples</i> in the 
</div>

In [ ]:
# Combined sample and labels by iterating over the obtained oracle
if display_output: 
    for i, row in enumerate(oracle['oracle']): print(sample_list[i].ljust(30) + str(row))

We observe that the sample `sqrt(-16)` triggers a bug in the calculator, whereas the sample `sqrt(4)` does not show unusual behavior. Of course, we want to know why the sample fails the program. In a typical use case, the developers of the calculator program would now try other input samples and evaluate if similar inputs also trigger the program's failure. Let's try some more input samples; maybe we can refine our understanding of why the calculator crashes:

In [ ]:
# Our guesses (maybe the failure is also in the cos or tan function?)
guess_samples = ['cos(-16)', 'tan(-16)', 'sqrt(-100)', 'sqrt(-20.23412431234123)']

# lets obtain the execution outcome for each of our guess
guess_oracle = execute_samples(guess_samples)

# lets show the results
if display_output: 
    for i, row in enumerate(guess_oracle['oracle']): print(guess_samples[i].ljust(30) + str(row))

It looks like the failure only occurs in the `sqrt` function, however, only for specific `x` values. We could now try other values for `x` and repeat the process. However, this would be highly time-consuming and not an efficient debugging technique for a larger and more complex test subject.

Wouldn't it be great if there was a tool that automatically does this for us? And this is exactly what _Alhazen_ is used for. It helps us explain why specific input files fail a program. 

<div class="alert alert-info">
[Info]: <i>Alhazen</i> is a tool that automatically learns the circumstances of program failure by associating syntactical features of sample inputs with the execution outcome. The produced explanations (in the form of a decision tree) help developers focus on the input space's relevant aspects.
</div>

# Implementing Alhazen

<hr/>
<div class="alert alert-success alertsuccess">
[Task]: Complete the missing functions of <i>Alhazen</i> and explain what input samples result in the calculator's failure.
</div>

To help you complete Alhazen, we have already provided you with a complete framework for feedback loop. Your goal is to implement the missing functions such that _Alhazen_ can learn the circumstances of program failure.

You are required to implement the following functions: `extract_existence`, `extract_numeric`, `collect_features`, `transform_grammar`, `train_tree`, and `generate_samples`.

Please follow the instructions in the individual notebooks for a more detailed documentation of the required functions.

The following code cell imports all functions from the other notebooks.

In [ ]:
# feature extraction
from ipynb.fs.full.Activity1_1_FeatureExtraction import extract_existence, extract_numeric, collect_features

# transfrom grammar
from ipynb.fs.full.Activity1_2_GrammarTransformation import transform_grammar

# learn decision tree 
from ipynb.fs.full.Activity2_DecisionTreeLearner import train_tree

# generate new input files
# from ipynb.fs.full.Activity4_GenerateSamples import generate_samples
from ipynb.fs.full.Activity4_GenerateSamples import generate_samples_advanced
from ipynb.fs.full.Activity4_GenerateSamples import generate_samples_random as generate_samples

<hr/>

Additionally to the feedback loop, we provide you with an implementation of the `execute_sample` and the `get_all_input_specifications` functions. Please look at the corresponding notebooks for a detailed description of how to use them.

In [ ]:
# extract features from the decision tree (provided by us)
from ipynb.fs.full.Activity3_RequirementExtraction import get_all_input_specifications

# execute samples (provided by us)
from ipynb.fs.defs.Activity5_ExecuteSamples import execute_samples

##  Alhazen Class
<hr/>

In [ ]:
from typing import List
import pandas

from fuzzingbook.Grammars import Grammar
from ipynb.fs.full.helper import OracleResult
from ipynb.fs.full.helper import show_tree

GENERATOR_TIMEOUT = 10 # timeout in seconds

class Alhazen:
    
    def __init__(self, initial_inputs: List[str],
                 grammar: Grammar,
                 max_iter: int = 10,
                 generator_timeout: int = 10):
        
        self._initial_inputs = initial_inputs
        self._grammar = grammar
        self._max_iter = max_iter
        self._previous_samples = None
        self._data = None
        self._trees = []
        self._generator_timeout = generator_timeout
        self._setup()
        
    def _setup(self):
        self._previous_samples = self._initial_inputs
        
        self._all_features = extract_existence(self._grammar) + extract_numeric(self._grammar)
        self._feature_names = [f.name for f in self._all_features]
        
    def run(self):
        raise NotImplementedError()
        
    def _add_new_data(self, exec_data, feature_data):
        joined_data = exec_data.join(feature_data.drop(['sample'], axis=1))
        
        # Only add valid data
        new_data = joined_data[(joined_data['oracle'] != OracleResult.UNDEF)]
        new_data = joined_data.drop(joined_data[joined_data.oracle.astype(str) == "UNDEF"].index)
        if 0 != len(new_data):
            if self._data is None:
                self._data = new_data
            else:
                self._data = pandas.concat([self._data, new_data], sort=False)
                
    def _finalize(self):
        return self._trees

In [ ]:
class Alhazen(Alhazen):
    
    def run(self):
        for iteration in range(self._max_iter):
            print(f"Starting Iteration: " + str(iteration))
            self._loop(self._previous_samples)
            
        return self._finalize()
        
    
    def _loop(self, sample_list):
        # obtain labels, execute samples (Initial Step, Activity 5)
        exec_data = execute_samples(sample_list)
        
        # collect features from the new samples (Activity 1)
        feature_data = collect_features(sample_list, self._grammar)
        
        # combine the new data with the already existing data
        self._add_new_data(exec_data, feature_data)
        
        # train a tree (Activity 2)
        dec_tree = train_tree(self._data)
        self._trees.append(dec_tree)
        
        # extract new requirements from the tree (Activity 3)
        new_input_specifications = get_all_input_specifications(dec_tree, 
                                                self._all_features, 
                                                self._feature_names, 
                                                self._data.drop(['oracle'], axis=1))
        
        # generate new inputs according to the new input specifications
        # (Activity 4)
        new_samples = generate_samples(self._grammar, new_input_specifications, self._generator_timeout)
        self._previous_samples = new_samples

</hr>

If you have correctly implemented the missing functions, we can finally run _Alhazen_!

In [ ]:
from fuzzingbook.Grammars import Grammar, is_valid_grammar
import string

HEARTBLEED: Grammar = {
    "<start>": ["<length>" " x" "<payload>" " y<padding>"],
    "<length>": ["<number>"],
    "<number>": ["<onenine><maybe_digits>"],
    "<onenine>": [str(num) for num in range(1, 10)],
    "<maybe_digits>": ["", "<digits>"],
    "<digits>": ["<digit>", "<digit><digits>"],
    "<digit>": list(string.digits),

    "<payload>": ["", "<digit><payload>"],
    "<padding>": ["", "<digit><padding>"]
}

assert is_valid_grammar(HEARTBLEED)

positive_inputs = ["6 x3 y13", "125 x4 y", "6512 x10 y2", "7992 x66 y39337874"]
negative_inputs = ["4 x875611395 y3", "12 x123456789101 y3"]

sample_list = positive_inputs + negative_inputs
sample_list

In [ ]:
# set the number of refinement iterations and the timeout for the input generator
# the execution time of Alhazen mainly depends on the number of iterations

GRAMMAR = HEARTBLEED
MAX_ITERATION = 20
GENERATOR_TIMEOUT = 10 # timeout in seconds

# let's initialize Alhazen
# let's use the previously used sample_list (['sqrt(-16)', 'sqrt(4)'])
alhazen = Alhazen(sample_list, GRAMMAR, MAX_ITERATION, GENERATOR_TIMEOUT)

# and run it
# Alhazen returns a list of all the iteratively learned decision trees
trees = alhazen.run()


</hr>

Let's display the final decision tree learned by Alhazen. You can use the function `show_tree(decison_tree, features)` to display the final tree.

In [ ]:
from src.requirementExtractionDT.treetools import remove_unequal_decisions

all_features = extract_existence(GRAMMAR) + extract_numeric(GRAMMAR)
# show_tree(trees[MAX_ITERATION-1], all_features)

<div class="alert alert-info" role="alert">
[Info] The decision tree may contain unneccesary long paths, where the bug-class does not change. You can use the function 'remove_unequal_decisions(decision_tree)' to remove those nodes.
</div>

In [ ]:
from src.requirementExtractionDT.treetools import remove_unequal_decisions

show_tree(remove_unequal_decisions(trees[MAX_ITERATION-1]), all_features)

You should now be able to identify the features that are responsible for the caluclator's failue!

`Real Solution`: The failure occurs whenever the function 'sqrt(x)' is used and x is between '-12' and '-42'

# Evaluation

<hr/>
Let's evaluate the learned classification model! We judge the quality of the learned decision tree learner by assessing its capabilities of predicting the behavior of newly generated inputs. 

## Evaluation Setup (Generating an Evaluation Data Set)

In the first step of evaluation of the learned classifier, we generate a evaluation data set.

In [ ]:
# We import the GrammarFuzzer
from fuzzingbook.GrammarFuzzer import GrammarFuzzer

evaluation_data = []

# And generate 1000 input samples
g = GrammarFuzzer(GRAMMAR)
for i in range(1000):
    evaluation_data.append(str(g.fuzz()))

In [ ]:
# Lets obtain the actuall program behavior of the evaluation data ['BUG', 'NO_BUG']
evaluation_exec_data = execute_samples(evaluation_data)
print(evaluation_exec_data) 

# Is the data set imbalanced? 
sample_bug_count = len(evaluation_exec_data[(evaluation_exec_data["oracle"].astype(str) == "BUG")])
sample_count = len(evaluation_exec_data)

print(f"{sample_bug_count} samples of {sample_count} generated inputs trigger the bug.")

In [ ]:
# let us obtain the features from the generated inputs
eval_feature_data = collect_features(evaluation_data, GRAMMAR)
# display(eval_feature_data)

In [ ]:
# Clean up the evaluation data
joined_data = evaluation_exec_data.join(eval_feature_data.drop(['sample'], axis=1))

# Only add valid data
new_data = joined_data[(joined_data['oracle'] != OracleResult.UNDEF)]
clean_data = joined_data.drop(joined_data[joined_data.oracle.astype(str) == "UNDEF"].index).fillna(0)

## Evaluation Results

<hr/>
Let's use the generated evaluation set to measure the accuracy, precision, recall and f1-score of your learned machine learning model.

<div class="alert alert-info" role="alert">
[Info] We use <a href="https://scikit-learn.org/stable/">scikit-learn</a> to evalute the classifier.
</div>

In [ ]:
eval_iteration = MAX_ITERATION - 1
final_tree = remove_unequal_decisions(trees[eval_iteration])

In [ ]:
# We use the final decision tree to predict the behavior of the evaluation data set.
predictions = final_tree.predict(clean_data.drop(['oracle'], axis = 1))

Let's measure the accuracy of the learned decision tree

<div class="alert alert-info" role="alert">
[Info] We start by measuering the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html">accuracy</a> of the classifier.
</div>

In [ ]:
# We calculate the accuracy by comparing how many predictions match the actual program behavior
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(clean_data['oracle'].astype(str), predictions, normalize=True)
# we round the accuracy to three digits
accuracy = round(accuracy*100, 3)
print(f"The decison tree at iteration {str(eval_iteration)} achieved an accuracy of {accuracy} %")

<div class="alert alert-info" role="alert">
[Info] We use the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html">precision-score</a> and the <a href="https://scikit-learn.org/stable/">recall-score</a>.
</div>

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

precision = precision_score(clean_data['oracle'].astype(str), predictions, pos_label='BUG', average='binary')
precision = round(precision*100, 3)
recall = recall_score(clean_data['oracle'].astype(str), predictions, pos_label='BUG', average='binary')
recall = round(recall*100, 3)

print(f"The decison tree at iteration {str(eval_iteration)} achieved a precision of {precision} %")
print(f"The decison tree at iteration {str(eval_iteration)} achieved a recall of {recall} %")

<div class="alert alert-info" role="alert">
[Info] To counteract the imbalanced data set, we use the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html">F1-score</a>.
</div>

In [ ]:
from sklearn.metrics import f1_score

f1 = f1_score(clean_data['oracle'].astype(str), predictions, pos_label='BUG', average='binary')
print(f"The decison tree at iteration {str(eval_iteration)} achieved a f1-score of {round(f1, 3)}")

## Congratulations

You did it, congratulations!